In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-small")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
def load_model(checkpoint_path):
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()
    print(f"Loaded model from {checkpoint_path}")
    return model


In [ ]:
def generate_summaries(model, texts, tokenizer, max_length=128, num_beams=4):
    summaries = []
    for text in texts:
        inputs = tokenizer(
            f"summarize: {text}", return_tensors="pt", truncation=True, padding="max_length", max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_length,
                num_beams=num_beams,
                early_stopping=True,
                length_penalty=1.2,
            )
            summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
            summaries.append(summary)

    return summaries


In [ ]:
def compare_models(model_paths, texts, tokenizer, max_length=128, num_beams=4):
    all_results = {}

    for path in model_paths:
        model = load_model(path)
        summaries = generate_summaries(model, texts, tokenizer, max_length=max_length, num_beams=num_beams)
        all_results[path] = summaries

    return all_results


In [ ]:
texts = [
    "Artificial intelligence is a branch of computer science that aims to create machines that mimic human intelligence. It encompasses subfields like machine learning, natural language processing, and robotics.",
    "Machine learning enables systems to learn and improve from data without being explicitly programmed. It is a crucial component of AI and has revolutionized industries like healthcare, finance, and autonomous vehicles."
]

model_paths = [
    "/kaggle/input/summarizer-t5-small-epoch1and5/transformers/default/1/checkpoint_epoch_1.pt",
    "/kaggle/input/summarizer-t5-small-epoch1and5/transformers/default/1/checkpoint_epoch_5.pt",
]

results = compare_models(model_paths, texts, tokenizer, max_length=128, num_beams=4)

for model_path, summaries in results.items():
    print(f"\nSummaries from model at {model_path}:")
    for i, summary in enumerate(summaries):
        print(f"Text {i + 1}: {texts[i][:100]}...") 
        print(f"Summary: {summary}\n")


/tmp/ipykernel_30/2028587808.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Loaded model from /kaggle/input/summarizer-t5-small-epoch1and5/transformers/default/1/checkpoint_epoch_1.pt
Loaded model from /kaggle/input/summarizer-t5-small-epoch1and5/transformers/default/1/checkpoint_epoch_5.pt

Summaries from model at /kaggle/input/summarizer-t5-small-epoch1and5/transformers/default/1/checkpoint_epoch_1.pt:
Text 1: Artificial intelligence is a branch of computer science that aims to create machines that mimic huma...
Summary: Learn about Artificial Intelligence.

Text 2: Machine learning enables systems to learn and improve from data without being explicitly programmed....
Summary: Use machine learning to learn from data.


Summaries from model at /kaggle/input/summarizer-t5-small-epoch1and5/transformers/default/1/checkpoint_epoch_5.pt:
Text 1: Artificial intelligence is a branch of computer science that aims to create machines that mimic huma...
Summary: Learn about artificial intelligence.

Text 2: Machine learning enables systems to learn and improve from data